In [9]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping

from scikeras.wrappers import KerasClassifier

from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix

import numpy as np
import pandas as pd

from joblib import dump, load

In [11]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    accuracy = accuracy_score(y_test, y_pred_classes)
    
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test, y_pred_classes, average='macro')

    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [5]:
x_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_test.npy")
x_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_train.npy")
x_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_val.npy")
y_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_test.npy")
y_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_train.npy")
y_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_val.npy")

In [6]:
input_shape = x_train.shape[1:]  # Determined by the shape of your training data
output_shape = len(np.unique(y_train))  # Assuming classification task

In [14]:
def create_model(units1=32, units2=28, units3=24, activation1='relu', activation2='relu', activation3='relu', lr=0.3):
    model = Sequential()
    model.add(Dense(units1, input_shape=input_shape, activation=activation1))
    model.add(Dense(units2, activation=activation2))
    model.add(Dense(units3, activation=activation3))
    model.add(Dense(output_shape, activation='softmax'))
    model.compile(optimizer=SGD(learning_rate=lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrapping the model with KerasClassifier
model = KerasClassifier(
    model=create_model,
    verbose = 1,
    units1 = [72, 64, 56],
    units2 = [64, 56, 48],
    units3 = [56, 48, 40],
    activation1 = ['relu', 'sigmoid', 'tanh'],
    activation2 = ['relu', 'sigmoid', 'tanh'],
    activation3 = ['relu', 'sigmoid', 'tanh'],
    lr = [0.3, 0.2, 0.1],
    batch_size = [32,64,128,256,512,1024],
    epochs = 20
)

# Define the parameter grid for hyperparameter tuning
param_dist = {
    'units1': [72, 64, 56],
    'units2': [64, 56, 48],
    'units3': [56, 48, 40],
    'activation1': ['relu', 'sigmoid', 'tanh'],
    'activation2': ['relu', 'sigmoid', 'tanh'],
    'activation3': ['relu', 'sigmoid', 'tanh'],
    'lr': [0.3, 0.2, 0.1],
    'batch_size': [32,64,128,256,512,1024]
}


random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, 
                                   n_iter=30, cv=2, verbose=2)

# Perform hyperparameter tuning using x_val and y_val
random_search_result = random_search.fit(x_val, y_val)

# Best parameters
best_params = random_search_result.best_params_
print(f"Best Parameters: {best_params}")

Fitting 2 folds for each of 30 candidates, totalling 60 fits
Epoch 1/20
1753/1753 [==============================] - 2s 733us/step - loss: 0.9038 - accuracy: 0.7488
Epoch 2/20
1753/1753 [==============================] - 1s 732us/step - loss: 0.4481 - accuracy: 0.8667
Epoch 3/20
1753/1753 [==============================] - 1s 718us/step - loss: 0.3434 - accuracy: 0.8975
Epoch 4/20
1753/1753 [==============================] - 1s 747us/step - loss: 0.2736 - accuracy: 0.9157
Epoch 5/20
1753/1753 [==============================] - 1s 740us/step - loss: 0.2174 - accuracy: 0.9326
Epoch 6/20
1753/1753 [==============================] - 1s 736us/step - loss: 0.2089 - accuracy: 0.9369
Epoch 7/20
1753/1753 [==============================] - 1s 716us/step - loss: 0.1807 - accuracy: 0.9438
Epoch 8/20
1753/1753 [==============================] - 1s 728us/step - loss: 0.1681 - accuracy: 0.9480
Epoch 9/20
1753/1753 [==============================] - 1s 726us/step - loss: 0.1679 - accuracy: 0.9481
Epo

In [15]:
# Train the optimal model with best parameters
optimal_model = create_model(
    units1=best_params['units1'],
    units2=best_params['units2'],
    units3=best_params['units3'],
    activation1=best_params['activation1'],
    activation2=best_params['activation2'],
    activation3=best_params['activation3'],
    lr=best_params['lr']
)
optimal_model.fit(x_train, y_train, batch_size=best_params['batch_size'], epochs=50, validation_split = 0.2,
                        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5)])

Epoch 1/50
11219/11219 [==============================] - 10s 868us/step - loss: 0.1951 - accuracy: 0.9413 - val_loss: 0.1226 - val_accuracy: 0.9624
Epoch 2/50
11219/11219 [==============================] - 10s 859us/step - loss: 0.1214 - accuracy: 0.9619 - val_loss: 0.1084 - val_accuracy: 0.9661
Epoch 3/50
11219/11219 [==============================] - 10s 864us/step - loss: 0.1113 - accuracy: 0.9648 - val_loss: 0.1021 - val_accuracy: 0.9676
Epoch 4/50
11219/11219 [==============================] - 10s 856us/step - loss: 0.1056 - accuracy: 0.9665 - val_loss: 0.1285 - val_accuracy: 0.9498
Epoch 5/50
11219/11219 [==============================] - 10s 857us/step - loss: 0.1016 - accuracy: 0.9675 - val_loss: 0.1023 - val_accuracy: 0.9680
Epoch 6/50
11219/11219 [==============================] - 10s 860us/step - loss: 0.0989 - accuracy: 0.9683 - val_loss: 0.1007 - val_accuracy: 0.9685
Epoch 7/50
11219/11219 [==============================] - 10s 868us/step - loss: 0.0962 - accuracy: 0.9690

In [7]:
#Elvin's model

def create_model(units1=32, units2=28, units3=24, activation1='relu', activation2='relu', activation3='relu'):
    model = Sequential()
    model.add(Dense(units1, input_shape=input_shape, activation=activation1))
    model.add(Dense(units2, activation=activation2))
    model.add(Dense(units3, activation=activation3))
    model.add(Dense(output_shape, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

optimal_model = create_model(
    units1=200,
    units2=200,
    units3=200,
    activation1='relu',
    activation2='relu',
    activation3='relu',
)
optimal_model.fit(x_train, y_train, batch_size=100, epochs=50, validation_split = 0.2,
                        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5)])

Epoch 1/50
3590/3590 [==============================] - 6s 2ms/step - loss: 0.1620 - accuracy: 0.9492 - val_loss: 0.1058 - val_accuracy: 0.9645
Epoch 2/50
3590/3590 [==============================] - 6s 2ms/step - loss: 0.0941 - accuracy: 0.9678 - val_loss: 0.0868 - val_accuracy: 0.9697
Epoch 3/50
3590/3590 [==============================] - 6s 2ms/step - loss: 0.0810 - accuracy: 0.9717 - val_loss: 0.0787 - val_accuracy: 0.9708
Epoch 4/50
3590/3590 [==============================] - 6s 2ms/step - loss: 0.0713 - accuracy: 0.9754 - val_loss: 0.0745 - val_accuracy: 0.9733
Epoch 5/50
3590/3590 [==============================] - 5s 1ms/step - loss: 0.0671 - accuracy: 0.9768 - val_loss: 0.0694 - val_accuracy: 0.9757
Epoch 6/50
3590/3590 [==============================] - 5s 2ms/step - loss: 0.0632 - accuracy: 0.9779 - val_loss: 0.0577 - val_accuracy: 0.9799
Epoch 7/50
3590/3590 [==============================] - 5s 1ms/step - loss: 0.0614 - accuracy: 0.9786 - val_loss: 0.0605 - val_accuracy:

In [80]:
# 1st tuning result n_iter = 10, cv = 3
calculate_performance_metrics(x_test, y_test, optimal_model)

4383/4383 [==============================] - 2s 541us/step
Accuracy: 0.9707337944804963
AUC: 0.9932935979494137

macro
Precision: 0.9537591471165549
Recall: 0.7641794973221735
F1 Score: 0.783915426401536

weighted
Precision: 0.9716198411274721
Recall: 0.9707337944804963
F1 Score: 0.9680806597136917

Mean FNR: 0.23582050267782648
Mean TNR: 0.9971426003680203
Mean FPR: 0.002857399631979744
Mean TPR: 0.7641794973221735


In [10]:
# 2nd tuning result n_iter = 10, cv = 3
calculate_performance_metrics(x_test, y_test, optimal_model)

4383/4383 [==============================] - 2s 550us/step
Accuracy: 0.9697639592098695
AUC: 0.996767241489942

macro
Precision: 0.9494536773176815
Recall: 0.8244192785590583
F1 Score: 0.8680281058529145

weighted
Precision: 0.9698579250459004
Recall: 0.9697639592098695
F1 Score: 0.967574351112461

Mean FNR: 0.17558072144094192
Mean TNR: 0.9971055040671141
Mean FPR: 0.0028944959328858883
Mean TPR: 0.8244192785590583


In [16]:
# 3rd tuning result n_iter = 30, cv = 2
calculate_performance_metrics(x_test, y_test, optimal_model)

4383/4383 [==============================] - 2s 550us/step
Accuracy: 0.9764458389788205
AUC: 0.9983211478568285

macro
Precision: 0.969054637979428
Recall: 0.8093328906067864
F1 Score: 0.8550217676954056

weighted
Precision: 0.9769818561966603
Recall: 0.9764458389788205
F1 Score: 0.9739949514293986

Mean FNR: 0.19066710939321355
Mean TNR: 0.9974291156281381
Mean FPR: 0.0025708843718620237
Mean TPR: 0.8093328906067864


In [6]:
# 4rd tuning result
calculate_performance_metrics(x_test, y_test, optimal_model)

4383/4383 [==============================] - 3s 560us/step
Accuracy: 0.9697354346430863
AUC: 0.9975596062042088

macro
Precision: 0.9539427301041344
Recall: 0.8284529308217552
F1 Score: 0.8658778245536036

weighted
Precision: 0.9704450229295069
Recall: 0.9697354346430863
F1 Score: 0.9675924779607082

Mean FNR: 0.17154706917824478
Mean TNR: 0.9971222410820255
Mean FPR: 0.002877758917974551
Mean TPR: 0.8284529308217552


In [12]:
# 5th tuning result
calculate_performance_metrics(x_test, y_test, optimal_model)

4383/4383 [==============================] - 3s 573us/step
Accuracy: 0.9813449333238251
AUC: 0.9990258499266507

macro
Precision: 0.9620231629260075
Recall: 0.8695763205198306
F1 Score: 0.9015009287398761

weighted
Precision: 0.9620231629260075
Recall: 0.8695763205198306
F1 Score: 0.9015009287398761

Mean FNR: 0.13042367948016953
Mean TNR: 0.9979873054042903
Mean FPR: 0.0020126945957096884
Mean TPR: 0.8695763205198306


In [7]:
dump(optimal_model, "/home/jovyan/X-IIoT/dl/dl_new/model/dnn.joblib")

INFO:tensorflow:Assets written to: ram://46761a0e6a8749229b29aff7892933ba/assets


INFO:tensorflow:Assets written to: ram://46761a0e6a8749229b29aff7892933ba/assets


['/home/jovyan/X-IIoT/dl/dl_new/model/dnn.joblib']